In [1]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import torch
import pickle
import json

In [ ]:
#Download captions tar
!wget https://dl.fbaipublicfiles.com/densely_captioned_images/dci.tar.gz

In [ ]:
#Untar the caption files. Please change the paths
!tar -xzvf "dci.tar.gz" -C "data/"

In [ ]:
#Download images tar file,m will download to the current directory.
!wget "https://scontent.xx.fbcdn.net/m1/v/t6/An_zz_Te0EtVC_cHtUwnyNKODapWXuNNPeBgZn_3XY8yDFzwHrNb-zwN9mYCbAeWUKQooCI9mVbwvzZDZzDUlscRjYxLKsw.tar?_nc_gid&ccb=10-5&oh=00_AfLvnV986h86zRzkguB36TyNtnY0ZO50Q8ja4V_8AQfA3Q&oe=685F618A&_nc_sid=0fdd51"

In [8]:
%mkdir images

In [ ]:
# Unzip the image tar into the new "images" into a path. Check path at the end
!tar -xzvf "An_zz_Te0EtVC_cHtUwnyNKODapWXuNNPeBgZn_3XY8yDFzwHrNb-zwN9mYCbAeWUKQooCI9mVbwvzZDZzDUlscRjYxLKsw.tar@_nc_gid&ccb=10-5&oh=00_AfLvnV986h86zRzkguB36TyNtnY0ZO50Q8ja4V_8AQfA3Q&oe=685F618A&_nc_sid=0fdd51" -C "images/"

In [2]:
# Change to relevant path, to the caption file (to densely_captioned_images/complete)
%ls
%cd data/densely_captioned_images/complete

image_name_2_caption = {}
listed_names = set()

for i, filename in enumerate(os.listdir()):
    file = open(filename)
    caption_data = json.load(file)
    file.close()
    listed_names.add(caption_data["image"])
    caption = caption_data["extra_caption"]
    image_name = caption_data["image"]
    image_name_2_caption[image_name] = caption

len(image_name_2_caption)

 Volume in drive C has no label.
 Volume Serial Number is 0899-657C

 Directory of C:\Users\Kate\Desktop\гзсЎ \г­ЁўҐа\projects\VLMs\DCI

19.06.2025  21:17    <DIR>          .
19.06.2025  18:13    <DIR>          ..
19.06.2025  18:14    <DIR>          .ipynb_checkpoints
19.06.2025  19:26    <DIR>          data
19.06.2025  20:57    <DIR>          images
19.06.2025  21:17           628я143 prep_embedded_data_densecaption.ipynb
               1 File(s)        628я143 bytes
               5 Dir(s)  12я994я699я264 bytes free
C:\Users\Kate\Desktop\учёба\универ\projects\VLMs\DCI\data\densely_captioned_images\complete


C:\Users\Kate\miniconda3\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


7805

In [3]:
# Check path
%cd ../../../images

C:\Users\Kate\Desktop\учёба\универ\projects\VLMs\DCI\images


In [4]:
data = [] # Data is a tuple of this format: (Pil.Image, text_caption)

for i, filename in enumerate(os.listdir()):
  if filename.endswith("jpg"):
    if filename not in listed_names:
      continue
    image = Image.open(filename)
    paired_caption = image_name_2_caption[filename]
    data.append((image, paired_caption))

In [5]:
assert len(data) == len(image_name_2_caption)

In [6]:
# load clip like normally.
from transformers import CLIPProcessor, CLIPModel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [7]:
# Consider removing this to save memory.
captions = [data[i][1] for i in range(len(data))]
images = [data[i][0] for i in range(len(data))]

# create batches of data
captions_batches = [captions[i:i+32] for i in range(0, len(captions), 32)]
images_batches = [images[i:i+32] for i in range(0, len(images), 32)]

# After this, consider free memory from these variables: data, captions, images



In [8]:
len(captions_batches)

244

In [9]:
import torch
import os

save_dir = "embeds_temp"
os.makedirs(save_dir, exist_ok=True)

# First pass: save each batch to disk
for i, (captions_local, images_local) in enumerate(zip(captions_batches, images_batches)):
    if i < 131:
        continue
        
    inputs = processor(text=captions_local, images=images_local, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(device, non_blocking=True) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        image_features = outputs.image_embeds.cpu()
        text_features = outputs.text_embeds.cpu()

    # Save to disk as a .pt file
    torch.save((image_features, text_features), os.path.join(save_dir, f"batch_{i:03d}.pt"))
    print(f"Saved batch {i}")

    # Optional memory cleanup
    del inputs, outputs, image_features, text_features
    torch.cuda.empty_cache()

Saved batch 131
Saved batch 132
Saved batch 133
Saved batch 134
Saved batch 135
Saved batch 136
Saved batch 137
Saved batch 138
Saved batch 139
Saved batch 140
Saved batch 141
Saved batch 142
Saved batch 143
Saved batch 144
Saved batch 145
Saved batch 146
Saved batch 147
Saved batch 148
Saved batch 149
Saved batch 150
Saved batch 151
Saved batch 152
Saved batch 153
Saved batch 154
Saved batch 155
Saved batch 156
Saved batch 157
Saved batch 158
Saved batch 159
Saved batch 160
Saved batch 161
Saved batch 162
Saved batch 163
Saved batch 164
Saved batch 165
Saved batch 166
Saved batch 167
Saved batch 168
Saved batch 169
Saved batch 170
Saved batch 171
Saved batch 172
Saved batch 173
Saved batch 174
Saved batch 175
Saved batch 176
Saved batch 177
Saved batch 178
Saved batch 179
Saved batch 180
Saved batch 181
Saved batch 182
Saved batch 183
Saved batch 184
Saved batch 185
Saved batch 186
Saved batch 187
Saved batch 188
Saved batch 189
Saved batch 190
Saved batch 191
Saved batch 192
Saved ba

In [15]:
embeds = []

for i in range(len(captions_batches)):
    image_features, text_features = torch.load(os.path.join(save_dir, f"batch_{i:03d}.pt"))
    print(len(image_features))
    embeds.extend(zip(image_features, text_features))

print("Reconstructed embeds:", len(embeds))


32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
29
Reconstructed embeds: 7805


In [16]:
len(captions_batches)

244

In [17]:
%cd ..
%ls

C:\Users\Kate\Desktop\учёба\универ\projects\VLMs\DCI


C:\Users\Kate\miniconda3\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


 Volume in drive C has no label.
 Volume Serial Number is 0899-657C

 Directory of C:\Users\Kate\Desktop\гзсЎ \г­ЁўҐа\projects\VLMs\DCI

19.06.2025  21:38    <DIR>          .
19.06.2025  18:13    <DIR>          ..
19.06.2025  18:14    <DIR>          .ipynb_checkpoints
19.06.2025  19:26    <DIR>          data
19.06.2025  20:57    <DIR>          images
19.06.2025  21:38           115я975 prep_embedded_data_densecaption.ipynb
               1 File(s)        115я975 bytes
               5 Dir(s)  16я112я513я024 bytes free


In [18]:
with open("dense_caption_embeds13-20", "wb") as fp:   #Pickling
    pickle.dump(embeds, fp)

embeds = []